In [2]:
# Import necessary libraries for basic functions
import numpy as np
import tkinter
import pandas as pd
import time
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import datetime as dt
from datetime import date, datetime, timedelta

# Import libraries for Stock, Currencies and Commodities Data extraction
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.foreignexchange import ForeignExchange
import yfinance as yf

# Import Libraries for Twitter Sentiment Analysis 
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import snscrape.modules.twitter as sntwitter

# Import Libraries for ML Algorithm
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn import neighbors
from sklearn.metrics import f1_score, accuracy_score

In [3]:
##Get Fortune-500 companies list from Wikipedia
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
df.to_csv('S&P500-Info.csv')
df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])

In [4]:
##Create GUI box to select Stock ticker
window = tkinter.Tk()

window.geometry("%dx%d+%d+%d" % (330, 80, 200, 150))
window.title("Fortune 500 Stocks")

# Dropdown menu options
dropdown = df['Symbol'].tolist()
dd_timeint = ('1min','5min','60min', '1day')

#updates text
def func(selected_item):
  print(repr(selected_item.strip()))

# Close window after selecting 2nd dropdown value
def func2(selected_item):
  print(repr(selected_item.strip()))
  window.destroy()  # close window

#create a dropdown list for Ticker Selection
var = tkinter.StringVar()
var.set('Ticker Name')
p = tkinter.OptionMenu(window, var, *dropdown, command=func)

p.pack()
display = tkinter.Label(window)
display.pack()

#create a dropdown list for Time Interval
var2 = tkinter.StringVar()
var2.set('Time Interval')
p = tkinter.OptionMenu(window, var2, *dd_timeint, command=func2)

p.pack()
display = tkinter.Label(window)
display.pack()

# on change dropdown value
def change_dropdown(*args):
    print( var.get() )
    print( var2.get() )

# link function to change dropdown
var.trace('w', change_dropdown)
var2.trace('w', change_dropdown)

window.mainloop()

AAPL
Time Interval
'AAPL'
AAPL
1day
'1day'


In [5]:
# Moving Average Class setup
class MovingAverage():
    def __init__(self, closingPrice):
        self.data = pd.DataFrame(closingPrice) 

# Calculating EMA
    def EMA(self, averaging_length=50):
        ret = self.data.ewm(
            span=averaging_length, 
            adjust=False).mean()
        return ret.rename(columns={'4. close': 'EMA'})

# Return Components of MACD
    def MACD(self, a=12, b=26, c=9):
        MACD_line = self.EMA(a) - self.EMA(b)
        signal_line = MACD_line.ewm(span=c, adjust=False).mean()
        histogram = MACD_line - signal_line
        return MACD_line, signal_line, histogram

In [6]:
def timeseriesfunc(ticker_name, data_interval):

    ## Use API key from Alpha Vantage
    api_key = 'T99XAOQH3CAGPE4V'

    ## Generate Alpha Vantage time series object
    ts = TimeSeries(key=api_key, output_format='pandas')
    app = ForeignExchange(key=api_key, output_format='pandas')
        
    ## Retrieve data for past 60 days
    if data_interval == '1day':
        #Fetch Stock daily data (past 60 days)
        data, meta_data = ts.get_daily(ticker_name, outputsize = 'compact') 
        
        #Fetch currency data (past 60 days)
        GBP_data, GBP_metadata = app.get_currency_exchange_daily(from_symbol = 'USD',to_symbol = 'GBP', outputsize = 'compact' )
        EUR_data, EUR_metadata = app.get_currency_exchange_daily(from_symbol = 'USD',to_symbol = 'EUR', outputsize = 'compact' )

        #Fetch Gold and oil daily data from Yahoo Finance
        if data_interval == '60min':
            intvl = data_interval[0:3]
        else:
            intvl = data_interval[0:2]

        comm = yf.download(['BTC-USD','GLD','CL=F'] , period="6mo", interval=intvl, auto_adjust=True)  
        comm = comm[['Close']]
        comm.columns = [col[1] for col in comm.columns]    
        Gold_CP = comm['GLD']
        Oil_CP = comm['CL=F']
    
    else:   
        data, meta_data = ts.get_intraday(ticker_name, interval = data_interval, outputsize = 'full')

        # #Fetch currency data (past 60 days)
        GBP_data, GBP_metadata = ts.get_intraday(symbol = "USDGBP", interval=data_interval, outputsize = 'full')
        EUR_data, EUR_metadata = ts.get_intraday(symbol = "USDEUR", interval=data_interval, outputsize = 'full')

        # #Fetch commodity data (past 60 days)
        GLD_data, GLD_metadata = ts.get_intraday(symbol = "GLD", interval=data_interval, outputsize = 'full') 
        Oil_data, Oil_metadata = ts.get_intraday(symbol = "CL", interval=data_interval, outputsize = 'full')

        Gold_CP = GLD_data['4. close']
        Oil_CP = Oil_data['4. close']


    GBP_CP = GBP_data['4. close']
    EUR_CP = EUR_data['4. close']

    data['date_time'] = data.index
    data['date_only'] = data['date_time'].dt.date

    # Calculate consecutive days price difference (Delta) for all - commodities,currencies and closing price of stock.
    ## Day1 - Day2
    Delta_GBP = GBP_CP.diff()
    Delta_EUR = EUR_CP.diff()
    Delta_Gold = Gold_CP.diff()
    Delta_Oil = Oil_CP.diff()

    ## Percentage changes in all fields
    OpeningPrice = data['1. open']
    Delta_OP = OpeningPrice.pct_change()

    high = data['2. high']
    Delta_high = high.pct_change()

    low = data['3. low']
    Delta_low = low.pct_change()
    
    closingPrice = data['4. close']
    Delta_CP = closingPrice.diff()
    
    volume = data['5. volume']
    Delta_Vol = volume.pct_change()
    
    mean = (( high + low ) / 2 )
    
    # Calculating Moving Average components
    MACD_indicator = MovingAverage(closingPrice)
    MACD_line, signal_line, histogram = MACD_indicator.MACD()

    # Convert data into dataframe
    df = pd.DataFrame(data)
    df.to_csv('Extracted Data before feature engineering.csv')

    # Add extra detailed columns (features) to the data
    df['Delta_OP'] = Delta_OP
    df['Delta_high'] = Delta_high
    df['Delta_low'] = Delta_low
    df['Delta_Vol'] = Delta_Vol
    df['Delta_CP'] = Delta_CP
    df['MeanPrice'] = mean
    df['MACD Line'] = MACD_line
    df['Signal Line'] = signal_line
    df['Histogram'] = histogram
    df['GBP_CP'] = GBP_CP     
    df['EUR_CP'] = EUR_CP  
    df['Gold_CP'] = Gold_CP
    df['Oil_CP'] = Oil_CP
    df['Delta_GBP'] = Delta_GBP     
    df['Delta_EUR'] = Delta_EUR  
    df['Delta_Gold'] = Delta_Gold
    df['Delta_Oil'] = Delta_Oil

    # Fill empty Data with the column Mean value
    df['Delta_EUR'].fillna((df['Delta_EUR'].mean()), inplace=True)
    df['Delta_GBP'].fillna((df['Delta_GBP'].mean()), inplace=True)
    df['Delta_Oil'].fillna((df['Delta_Oil'].mean()), inplace=True)
    df['Delta_Gold'].fillna((df['Delta_Gold'].mean()), inplace=True)    
    df['Delta_CP'].fillna((df['Delta_CP'].mean()), inplace=True)
    df['Delta_OP'].fillna((df['Delta_OP'].mean()), inplace=True)
    df['Delta_high'].fillna((df['Delta_high'].mean()), inplace=True)
    df['Delta_low'].fillna((df['Delta_low'].mean()), inplace=True)
    df['Delta_Vol'].fillna((df['Delta_Vol'].mean()), inplace=True)

    # Replace the difference with 0 or 1 depending upon their negative or positive value
    ## One-hot encoding
    df.loc[df.Delta_CP > 0, "Delta_CP"] = 1
    df.loc[df.Delta_CP < 0, "Delta_CP"] = 0

    df.loc[df.Delta_GBP > 0, "Delta_GBP"] = 1
    df.loc[df.Delta_GBP < 0, "Delta_GBP"] = 0

    df.loc[df.Delta_EUR > 0, "Delta_EUR"] = 1
    df.loc[df.Delta_EUR < 0, "Delta_EUR"] = 0

    df.loc[df.Delta_Gold > 0, "Delta_Gold"] = 1
    df.loc[df.Delta_Gold < 0, "Delta_Gold"] = 0

    df.loc[df.Delta_Oil > 0, "Delta_Oil"] = 1
    df.loc[df.Delta_Oil < 0, "Delta_Oil"] = 0

    #Copy commodities data into new column for addition of old data columns
    df['old_gold_1'] = df['Delta_Gold']
    df['old_oil_1'] = df['Delta_Oil']
    df['old_GBP_1'] = df['Delta_GBP']
    df['old_EUR_1'] = df['Delta_EUR']

    df['old_gold_2'] = df['Delta_Gold']
    df['old_oil_2'] = df['Delta_Oil']
    df['old_GBP_2'] = df['Delta_GBP']
    df['old_EUR_2'] = df['Delta_EUR']

    df['old_gold_3'] = df['Delta_Gold']
    df['old_oil_3'] = df['Delta_Oil']
    df['old_GBP_3'] = df['Delta_GBP']
    df['old_EUR_3'] = df['Delta_EUR']

    # Shift column up by 1 days (correlation with 1 day older data)
    df.old_gold_1 = df.old_gold_1.shift(-1)
    df.old_oil_1 = df.old_oil_1.shift(-1)
    df.old_GBP_1 = df.old_GBP_1.shift(-1)
    df.old_EUR_1 = df.old_EUR_1.shift(-1)
    df['old_gold_1'] = df['old_gold_1'].fillna(df['Delta_Gold'])
    df['old_oil_1'] = df['old_oil_1'].fillna(df['Delta_Oil'])
    df['old_GBP_1'] = df['old_GBP_1'].fillna(df['Delta_GBP'])
    df['old_EUR_1'] = df['old_EUR_1'].fillna(df['Delta_EUR'])

    # Shift column up by 2 days (correlation with 2 day older data)
    df.old_gold_2 = df.old_gold_2.shift(-2)
    df.old_oil_2 = df.old_oil_2.shift(-2)
    df.old_GBP_2 = df.old_GBP_2.shift(-2)
    df.old_EUR_2 = df.old_EUR_2.shift(-2)
    df['old_gold_2'] = df['old_gold_2'].fillna(df['Delta_Gold'])
    df['old_oil_2'] = df['old_oil_2'].fillna(df['Delta_Oil'])
    df['old_GBP_2'] = df['old_GBP_2'].fillna(df['Delta_GBP'])
    df['old_EUR_2'] = df['old_EUR_2'].fillna(df['Delta_EUR'])

    # Shift column up by 3 days (correlation with 3 day older data)
    df.old_gold_3 = df.old_gold_3.shift(-3)
    df.old_oil_3 = df.old_oil_3.shift(-3)
    df.old_GBP_3 = df.old_GBP_3.shift(-3)
    df.old_EUR_3 = df.old_EUR_3.shift(-3)
    df['old_gold_3'] = df['old_gold_3'].fillna(df['Delta_Gold'])
    df['old_oil_3'] = df['old_oil_3'].fillna(df['Delta_Oil'])
    df['old_GBP_3'] = df['old_GBP_3'].fillna(df['Delta_GBP'])
    df['old_EUR_3'] = df['old_EUR_3'].fillna(df['Delta_EUR'])

    # Add features from the twitter
    if __name__ == "__main__":
        # Checking Twitter Sentiments
        query = ["Gold OR gold OR crude OR Oil OR USD OR Dollar OR EUR OR Euro OR GBP OR Pound OR Great Britain Pound"]
        tweet_df = main(query, df)

        if len(tweet_df) != 0:
            df = df.join(tweet_df, how='outer', on='date_only')
            df['negative'].fillna((df['negative'].mean()), inplace=True)    
            df['neutral'].fillna((df['neutral'].mean()), inplace=True)
            df['positive'].fillna((df['positive'].mean()), inplace=True)
            df['pos_perc'].fillna((df['pos_perc'].mean()), inplace=True)
            df['neg_perc'].fillna((df['neg_perc'].mean()), inplace=True)
            df['neut_perc'].fillna((df['neut_perc'].mean()), inplace=True)  
            df['total_tw'].fillna((df['total_tw'].mean()), inplace=True)  
            df['tw_weight'].fillna((df['tw_weight'].mean()), inplace=True) 

    df = df[:-3]
    
    # Data extraction and feature engeering completed.
    ## Write the final dataset as an excel file into device
    if data_interval == '1min':
        df.to_csv("Final_Dataset_1min.csv")
    elif data_interval == '5min':
        df.to_csv("Final_Dataset_5min.csv")
    elif data_interval == '60min':
        df.to_csv("Final_Dataset_1hr.csv")
    elif data_interval == '1day':        
        df.to_csv("Final_Dataset_1day.csv")
    
    return df

In [7]:
def cls_threshold(dframe):

    df = pd.DataFrame(dframe)

    f1_gold = f1_score(df.Delta_CP, df.Delta_Gold, average=None)
    f1_oil = f1_score(df.Delta_CP, df.Delta_Oil, average=None)
    f1_gbp = f1_score(df.Delta_CP, df.Delta_GBP, average=None)
    f1_eur = f1_score(df.Delta_CP, df.Delta_EUR, average=None)

    f1_old_gold_1 = f1_score(df.Delta_CP, df.old_gold_1, average=None)
    f1_old_oil_1 = f1_score(df.Delta_CP, df.old_oil_1, average=None)
    f1_old_gbp_1 = f1_score(df.Delta_CP, df.old_GBP_1, average=None)
    f1_old_eur_1 = f1_score(df.Delta_CP, df.old_EUR_1, average=None)

    f1_old_gold_2 = f1_score(df.Delta_CP, df.old_gold_2, average=None)
    f1_old_oil_2 = f1_score(df.Delta_CP, df.old_oil_2, average=None)
    f1_old_gbp_2 = f1_score(df.Delta_CP, df.old_GBP_2, average=None)
    f1_old_eur_2 = f1_score(df.Delta_CP, df.old_EUR_2, average=None)

    f1_old_gold_3 = f1_score(df.Delta_CP, df.old_gold_3, average=None)
    f1_old_oil_3 = f1_score(df.Delta_CP, df.old_oil_3, average=None)
    f1_old_gbp_3 = f1_score(df.Delta_CP, df.old_GBP_3, average=None)
    f1_old_eur_3 = f1_score(df.Delta_CP, df.old_EUR_3, average=None)
    
    # Positive predictions (F1-Score) mean for calculation of Threshold Value
    threshold = (( f1_gold + f1_oil + f1_gbp + f1_eur ) / 4 )
    threshold1 = (( f1_old_gold_1 + f1_old_oil_1 + f1_old_gbp_1 + f1_old_eur_1 ) / 4 )
    threshold2 = (( f1_old_gold_2 + f1_old_oil_2 + f1_old_gbp_2 + f1_old_eur_2 ) / 4 )
    threshold3 = (( f1_old_gold_3 + f1_old_oil_3 + f1_old_gbp_3 + f1_old_eur_3 ) / 4 )

    if f1_gbp[0] < threshold[0]:
        df.drop(['Delta_GBP'], axis='columns', inplace=True) #drop column Delta_GBP
    if f1_eur[0] < threshold[0]:
        df.drop(['Delta_EUR'], axis='columns', inplace=True) #drop column Delta_EUR     
    if f1_gold[0] < threshold[0]:
        df.drop(['Delta_Gold'], axis='columns', inplace=True) #drop column Delta_Gold
    if f1_oil[0] < threshold[0]:
        df.drop(['Delta_Oil'], axis='columns', inplace=True) #drop column Delta_Oil

    if f1_old_gold_1[0] < threshold1[0]:
        df.drop(['old_gold_1'], axis='columns', inplace=True) #drop column old_gold_1
    if f1_old_oil_1[0] < threshold1[0]:
        df.drop(['old_oil_1'], axis='columns', inplace=True) #drop column old_oil_1
    if f1_old_gbp_1[0] < threshold1[0]:
        df.drop(['old_GBP_1'], axis='columns', inplace=True) #drop column old_GBP_1
    if f1_old_eur_1[0] < threshold1[0]:
        df.drop(['old_EUR_1'], axis='columns', inplace=True) #drop column old_EUR_1

    if f1_old_gold_2[0] < threshold2[0]:
        df.drop(['old_gold_2'], axis='columns', inplace=True) #drop column old_gold_2
    if f1_old_oil_2[0] < threshold2[0]:
        df.drop(['old_oil_2'], axis='columns', inplace=True) #drop column old_oil_2  
    if f1_old_gbp_2[0] < threshold2[0]:
        df.drop(['old_GBP_2'], axis='columns', inplace=True) #drop column old_GBP_2
    if f1_old_eur_2[0] < threshold2[0]:
        df.drop(['old_EUR_2'], axis='columns', inplace=True) #drop column old_EUR_2

    if f1_old_gold_3[0] < threshold3[0]:
        df.drop(['old_gold_3'], axis='columns', inplace=True) #drop column old_gold_3
    if f1_old_oil_3[0] < threshold3[0]:
        df.drop(['old_oil_3'], axis='columns', inplace=True) #drop column old_oil_3
    if f1_old_gbp_3[0] < threshold3[0]:
        df.drop(['old_GBP_3'], axis='columns', inplace=True) #drop column old_GBP_3
    if f1_old_eur_3[0] < threshold3[0]:
        df.drop(['old_EUR_3'], axis='columns', inplace=True) #drop column old_EUR_3         

In [8]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = 'klEqcFpgzV45QseGqY8bF0YUz'
        consumer_secret = 'Nr1Au9Va4DiFFPUYWeQvmW2Wc8r5iyVr392eeXmtFgU10NOhWf'
        access_token = '1412996250098614275-UGgVUXN72VYbguAdCBAaWQXEazgoZr'
        access_token_secret = 'wu29wfJgGRIhBlywCVxmGdqR7kdSoSnMjLRcPjxuLrcxx'

        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")

    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])(\w+:\/\/\S+)", " ", tweet).split())

    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def get_tweets(self, query, count, dfdate):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
        
        dfdate = dfdate.replace(' ', '')
        sdate = datetime.strptime(dfdate, "%Y-%m-%d")
        udate = sdate + timedelta(days=1)
        udate = udate.date()

        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = 100,  since = sdate, until=udate)

            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}

                day = pd.Timestamp(sdate)

                # saving text of tweet
                parsed_tweet['text'] = tweet.text

                #Add weekend tweets to Monday (Set Monday date for weekends to accumulate later)
                if day.dayofweek == 5:
                    parsed_tweet['date_only'] = tweet.created_at.date() + timedelta(days=2)
                elif day.dayofweek == 6:
                    parsed_tweet['date_only'] = tweet.created_at.date() + timedelta(days=1)
                else:
                    parsed_tweet['date_only'] = tweet.created_at.date()

                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)

            # return parsed tweets
            return tweets

        except tweepy.TweepError as e:
            TwitterError = e

def main(tweet_obj, df):
    # creating object of TwitterClient Class
    api = TwitterClient()

    # empty list to store all the tweets
    tweets_all = []
    dfdate = []

    dfdates = df['date_time'].astype('datetime64[ns]')

    for date in dfdates:
        date=pd.to_datetime(date).date()
        dfdate.append(date)

    dfdates = pd.DataFrame(dfdate)
    dfdates = dfdates.drop_duplicates(keep='last')
    # calling function to get tweets
    for i, date in dfdates.iterrows():
        date1 = str(date)
        date1 = date1[1:15]
        
        tweets = api.get_tweets(query = tweet_obj, count = 100, dfdate = date1 )
        if tweets:
            tweets_all.extend(tweets)

        #Check for day, of Friday: Find tweets from the weekend as well:
        day = pd.Timestamp(date1)
        if day.dayofweek == 4:
            saturday = date + timedelta(days=1)
            sat = str(saturday)
            sat = sat[1:15]
            sunday   = saturday + timedelta(days=1)
            sun = str(sunday)
            sun = sun[1:15]

            tweets = api.get_tweets(query = tweet_obj, count = 100, dfdate = sat )
            if tweets:
                tweets_all.extend(tweets)

            tweets = api.get_tweets(query = tweet_obj, count = 100, dfdate = sun )
            if tweets:
                tweets_all.extend(tweets)         

        # calling function to get tweets
        tweets_df = pd.DataFrame(tweets_all)
        tweets_df.to_csv("tweets.csv")

    if len(tweets_df) != 0:  
        tweets_df.sort_values(by=['date_only','sentiment'], ascending=False, inplace=True)
        tweets_df = tweets_df[['date_only','sentiment']]

        tweets_df = tweets_df.groupby(['date_only','sentiment'], as_index=False).size()
        tweets_df = tweets_df.pivot(index='date_only', columns='sentiment', values='size')

        tweets_df['total_tw'] = (tweets_df['positive'] + tweets_df['negative'] + tweets_df['neutral'] )
        tweets_df['pos_perc'] = (tweets_df['positive']/tweets_df['total_tw'])*100
        tweets_df['neg_perc'] = (tweets_df['negative']/tweets_df['total_tw'])*100
        tweets_df['neut_perc'] = (tweets_df['neutral']/tweets_df['total_tw'])*100
        tweets_df['tw_weight'] = tweets_df['positive'] - tweets_df['negative']
        tweets_df.to_csv("TwitterFeatures.csv")
    else:
        print('Twitter API limit exceeded. Please try after sometime to have twitter sentiments as features.')
    return tweets_df

In [9]:
def plot_delta_data(df, x_variable, y_variable, title):
    plt.figure(figsize=(25, 25), dpi=80, facecolor = 'w', edgecolor = 'k')
    plt.plot(df, x_variable, color='blue', label='Delta Opening Price')
    plt.plot(df, y_variable, color = 'red', label = 'Delta Closing Price')

    plt.title(title, fontsize = 40)
    plt.xlabel('Time', fontsize=40)
    plt.ylabel('Price', fontsize = 40)
    plt.legend(loc = 'best')
    plt.grid()
    plt.show()

def plot_data(df, x_variable, y_variable, title):
    plt.figure(figsize=(25, 25), dpi=80, facecolor = 'w', edgecolor = 'k')
    plt.plot(df, x_variable, color='blue', label='Opening Price')
    plt.plot(df, y_variable, color = 'red', label = 'Closing Price')

    plt.title(title, fontsize = 40)
    plt.xlabel('Time', fontsize=40)
    plt.ylabel('Price', fontsize = 40)
    plt.legend(loc = 'best')
    plt.grid()
    plt.show()

def plot_results(data):

    data.plot.bar(x="Model", y="RMSE", rot=0, title= var.get() + " Model Comparison")
    plt.ylabel("Root mean squared error (RMSE)")
    plt.grid()
    plt.show(block=True)

def plot_models(df, x_variable, y_variable, title):
    plt.figure(figsize=(25, 25), dpi=80, facecolor = 'w', edgecolor = 'k')

    plt.plot(df, x_variable, color='blue', label='Real Closing Price')
    plt.plot(df, y_variable, color = 'red', label = 'Predicted Closing Price')

    plt.title(title, fontsize = 30)
    plt.xlabel('Time', fontsize=20)
    plt.ylabel('Stock Closing Price', fontsize = 20)
    plt.legend(loc = 'best')
    plt.grid()
    plt.show()

In [10]:
# ML Algorithm Code
def algorithm(data):
    outputlist = []
    output_dataset = []
    df_out = pd.DataFrame(output_dataset)

    df_date = data['date_only']

    #Drop extra columns
    data.reset_index(drop=True, inplace=True)
    data = data.drop(data.columns[[5, 6, 16, 17, 18, 19]], axis=1) 
    # data = data.drop('total_tw')

    #Drop columns if commodity does not affect stock price  
    commodity_threshold = cls_threshold(data)
    # data = data.astype(int)

    #split dataset in features and target variable
    feature_cols = list(data.columns)
    target_var = ['4. close']

    X = data[feature_cols] # Features
    y = data[target_var] # Target variable

    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test
    num = len(X_test)
    df_date = df_date[:num]

    xtest_open = X_test['1. open']
    df_out['StockOpeningPrice'] = xtest_open

    df_out['Expected_CP'] = y_test

    # Decision Tree
    model = DecisionTreeRegressor(max_depth=5)
    model = model.fit(X_train,y_train)
    y_pred = model.predict(X_test) #Predicted Closing price of the stock
    desTr_Acc = metrics.mean_absolute_percentage_error(y_test, y_pred)
    #Update in excel
    df_out['DecTree_CP_Pred'] = y_pred
    plot_models(df = df_date,
            x_variable = y_test, 
            y_variable = y_pred, 
            title = var.get() + ":Decision Tree")

    # saving the output
    output = {}
    output['Model'] = 'DecisionTree'
    output['RMSE'] = desTr_Acc
    outputlist.append(output)

    #Knn Regression
    model = neighbors.KNeighborsRegressor()
    model = model.fit(X_train,y_train)                
    y_pred = model.predict(X_test) #Predicted Closing price of the stock
    knn_Acc = metrics.mean_absolute_percentage_error(y_test, y_pred)
    #Update in excel
    df_out['knn_CP_Pred'] = y_pred
    plot_models(df = df_date,
            x_variable = y_test, 
            y_variable = y_pred, 
            title = var.get() + ":KnnRegression")

    # saving the output
    output = {}
    output['Model'] = 'knnRegres'
    output['RMSE'] = knn_Acc
    outputlist.append(output)    

    #Random Forest
    ##grid = {'n_estimators': [200], 'max_depth': [3], 'max_features': [4, 8], 'random_state': [42]}
    model=RandomForestRegressor(n_estimators=100) #(grid)
    model = model.fit(X_train,y_train.values.ravel())
    y_pred = model.predict(X_test) #Predicted Closing price of the stock
    RanForr_Acc = metrics.mean_absolute_percentage_error(y_test, y_pred)    
    #Update in excel
    df_out['RanFor_CP_Pred'] = y_pred
    plot_models(df = df_date,
            x_variable = y_test, 
            y_variable = y_pred, 
            title = var.get() + ":Random Forest Regressor")

    # saving the output
    output = {}
    output['Model'] = 'RandomForest'
    output['RMSE'] = RanForr_Acc
    outputlist.append(output) 

    #Support Vector Regressor
    X = X_train
    y = y_train

    sc_X = StandardScaler()
    sc_y = StandardScaler()
    X = sc_X.fit_transform(X)
    y = sc_y.fit_transform(y)

    regressor = SVR(kernel = 'rbf')#'sigmoid')#'poly')
    regressor.fit(X, y)
    y_pred = regressor.predict(X_test)
    y_pred = sc_y.inverse_transform(y_pred)
    SVR_Acc = metrics.mean_absolute_percentage_error(y_test, y_pred)

    #Update in excel
    df_out['SVR_CP_Pred'] = y_pred
    plot_models(df = df_date,
            x_variable = y_test, 
            y_variable = y_pred, 
            title = var.get() + ":Support Vector Regressor")

    # saving the output
    output = {}
    output['Model'] = 'SVR'
    output['RMSE'] = SVR_Acc
    outputlist.append(output)

    #Find profit or loss
    df_out['DT_Analysis'] = ''
    df_out.loc[df_out.DecTree_CP_Pred > df_out.StockOpeningPrice, "DT_Analysis"] = 'Profit'
    df_out.loc[df_out.DecTree_CP_Pred < df_out.StockOpeningPrice, "DT_Analysis"] = 'Loss'

    df_out['Knn_Analysis'] = ''    
    df_out.loc[df_out.knn_CP_Pred > df_out.StockOpeningPrice, "Knn_Analysis"] = 'Profit'
    df_out.loc[df_out.knn_CP_Pred < df_out.StockOpeningPrice, "Knn_Analysis"] = 'Loss'

    df_out['RF_Analysis'] = ''
    df_out.loc[df_out.RanFor_CP_Pred > df_out.StockOpeningPrice, "RF_Analysis"] = 'Profit'
    df_out.loc[df_out.RanFor_CP_Pred < df_out.StockOpeningPrice, "RF_Analysis"] = 'Loss'

    df_out['SVR_Analysis'] = ''
    df_out.loc[df_out.SVR_CP_Pred > df_out.StockOpeningPrice, "SVR_Analysis"] = 'Profit'
    df_out.loc[df_out.SVR_CP_Pred < df_out.StockOpeningPrice, "SVR_Analysis"] = 'Loss'

    df_out['Expected_Analysis'] = ''
    df_out.loc[df_out.Expected_CP > df_out.StockOpeningPrice, "Expected_Analysis"] = 'Profit'
    df_out.loc[df_out.Expected_CP < df_out.StockOpeningPrice, "Expected_Analysis"] = 'Loss'    

    df_out.to_csv("AlgorithmResults.csv")

    #Best Model Analysis
    dt_f1 = f1_score(df_out.Expected_Analysis, df_out.DT_Analysis, average=None)
    knn_f1 = f1_score(df_out.Expected_Analysis, df_out.Knn_Analysis, average=None)
    rf_f1 = f1_score(df_out.Expected_Analysis, df_out.RF_Analysis, average=None)
    svr_f1 = f1_score(df_out.Expected_Analysis, df_out.SVR_Analysis, average=None)

    max_f1 = (max(dt_f1[0], knn_f1[0], rf_f1[0], svr_f1[0]))

    dt_acc = accuracy_score(df_out.Expected_Analysis , df_out.DT_Analysis) * 100
    knn_acc = accuracy_score(df_out.Expected_Analysis, df_out.Knn_Analysis) * 100
    rf_acc = accuracy_score(df_out.Expected_Analysis, df_out.RF_Analysis) * 100
    svr_acc = accuracy_score(df_out.Expected_Analysis, df_out.SVR_Analysis) * 100

    if max_f1 == dt_f1[0]:
        best_model = 'DecisionTree'
    elif max_f1 == knn_f1[0]:
        best_model = 'K-NearestNeighbors'
    elif max_f1 == rf_f1[0]:
        best_model = 'RandomForest'
    elif max_f1 == svr_f1[0]:
        best_model = 'SupportVectorRegressor'

    output = pd.DataFrame(outputlist)
    output.insert(loc=2, column = 'F1-Score', value = [dt_f1[0], knn_f1[0], rf_f1[0], svr_f1[0]])
    output.insert(loc=3, column = 'Accuracy', value = [dt_acc, knn_acc, rf_acc, svr_acc])
    print(output)

    # Note - RMSE is inversely proportional to F1-Score
    print('Best Model is: ' + best_model + ' with F1-Score: ' + str(max_f1) )
    plot_results(output) 

    return data, best_model, X_train, y_train, df_date

In [11]:
def predict(data, best_model, X_train, y_train, dfdate):

    today = date.today()
    dfdate = dfdate.shift()
    dfdate.iloc[0] = today
    dfdate = dfdate[:4]

    data = data.shift()
    data.iloc[0] = data.iloc[1]
    data['1. open'].iloc[0] = data['4. close'].iloc[1]

    if best_model == 'DecisionTree':
        model = DecisionTreeRegressor(max_depth=5)

    elif best_model == 'K-NearestNeighbors':
        model = neighbors.KNeighborsRegressor()

    elif best_model == 'RandomForest':
        model=RandomForestRegressor(n_estimators=100)
    
    test_rows = data[:4]

    model = model.fit(X_train,y_train)
    CP_Pred = model.predict(test_rows)

    test_rows['Predicted'] = CP_Pred
    test_rows = test_rows[['1. open', '4. close','Predicted']]
    test_rows.to_csv("Prediction.csv")

    CP_actual = test_rows['4. close']

    #Compare current date's opening and closing price and check for profit or loss
    if test_rows['1. open'].iloc[0] > test_rows['Predicted'].iloc[0]:
        print("Loss: Stock Opening Price is higher than the predicted Closing Price.")
    else:
        print("Profit: Predicted Closing Price is higher than the Stock Opening Price.")

    #Plot Graph comparing actual and predicted closing price for past 4 days
    plot_models(df = dfdate,
            x_variable = CP_actual, 
            y_variable = CP_Pred, 
            title ="Actual and " + var.get() +  " Predicted closing price for past 4 days Comparison")

In [ ]:
# Data Extraction, Feature Enginnering and Time-Series application
ts_data = timeseriesfunc(var.get(),var2.get())

## Plot Opening Price, Closing Price and their Delta values
df_date = ts_data['date_only']
x_delta = ts_data['Delta_OP']
y_delta = ts_data['Delta_CP']
x = ts_data['1. open']
y = ts_data['4. close']

plot_data(df = df_date,
        x_variable = x, 
        y_variable = y, 
        title = "Prices of " + var.get() + " : " + var2.get() + " Interval")

plot_delta_data(df = df_date,
        x_variable = x_delta, 
        y_variable = y_delta, 
        title = "Delta Prices of " + var.get() + " : " + var2.get() + " Interval")

# # Candlestick Graph
df1 = pd.DataFrame(ts_data)
fig = go.Figure(data=[go.Candlestick(x=df1["date_time"],
                open=df1['1. open'],
                high=df1['2. high'],
                low=df1['3. low'],
                close=df1['4. close'])])
fig.update_layout(title= var.get() + " : " + var2.get() + " Interval")
fig.show()

## ML Algorithms Application
data, best_model, X_train, y_train, dfdate = algorithm(ts_data)

## Prediction of the Stock Price movement
predict(data, best_model, X_train, y_train, dfdate)